In [ ]:
import pandas as pd
import pyodbc

In [ ]:
raw_data = pd.read_excel('uszips.xlsx')

In [ ]:
raw_data.head()

Here I connect to Databse to make more convenient way of solving the problem from jupyter notebook


In [ ]:
import os
import psycopg2
from dotenv import load_dotenv

load_dotenv()

connection = psycopg2.connect(host=os.environ.get('PG_HOST'),
                        port=os.environ.get('PG_PORT'),
                        user=os.environ.get('PG_USER'),
                        password=os.environ.get('PG_PASSWORD'),
                        dbname=os.environ.get('PG_DATABASE'),
                        sslmode='require')
connection.autocommit = True
cursor = connection.cursor()
cursor.execute('SELECT %s as connected;', ('Connection to postgres successful!',))
print(cursor.fetchone())

In [ ]:
createTableQuery = """
    CREATE TABLE IF NOT EXISTS my_table(
      id BIGSERIAL PRIMARY KEY NOT NULL ,
      name varchar,
      date TIMESTAMP NOT NULL DEFAULT current_timestamp
    );
  """
cursor.execute(createTableQuery)
#Just testing the databse response

In [ ]:
delete_table_query = """
    DROP TABLE IF EXISTS my_table;
  """
cursor.execute(delete_table_query)

In [ ]:
# craete_database = """
#     CREATE DATABASE narek_meloyan_hw1
# """
# cursor.execute(craete_database)

In [ ]:
city_table = raw_data[['city', 'state_id', 'state_name', 'population', 'density']].drop_duplicates()

In [ ]:
city_table

In [ ]:
import numpy as np
city_id = np.arange(1, city_table.shape[0]+1)

In [ ]:
city_table['city_id'] = city_id

In [ ]:
city_table

In [ ]:
create_city_table_query = f"""
    CREATE TABLE cities (
        city_id INT PRIMARY KEY NOT NULL,
        city_name VARCHAR(50) NOT NULL,
        state_id VARCHAR(2) NOT NULL,
        state_name VARCHAR(50) NOT NULL,
        pop FLOAT,
        density FLOAT
    );
    """
cursor = connection.cursor()
cursor.execute(create_city_table_query)

In [ ]:
def insert_values():
    for i in range(len(city_table)):
        city_name  = city_table.iloc[i, 0]
        state_id   = city_table.iloc[i, 1]
        state_name = city_table.iloc[i, 2]
        population = city_table.iloc[i, 3] if str(city_table.iloc[i, 3]) != 'nan' else "NULL"
        density    = city_table.iloc[i, 4] if str(city_table.iloc[i, 4]) != 'nan' else "NULL"
        city_id    = city_table.iloc[i, 5]
        insert_query = f"""
            INSERT INTO cities(city_id, city_name, state_id, state_name, pop, density)
            VALUES({int(city_id)}, '{city_name.replace("'", "''")}', '{state_id}', '{state_name}', {population}, {density})
        """
        cursor.execute(insert_query)
insert_values()

In [ ]:
city_id_data = raw_data.merge(city_table, left_on=['city', 'state_id', 'state_name', 'population', 'density'], right_on=['city', 'state_id', 'state_name', 'population', 'density'])
city_id_data

In [ ]:
zip_frame = city_id_data[['zip', 'lat', 'lng', 'city_id']]

In [ ]:
zip_frame.head()

In [ ]:
create_zip_table_query = f"""
    CREATE TABLE zip (
        zip INT PRIMARY KEY NOT NULL,
        latitude FLOAT(7) NOT NULL,
        longitude FLOAT(7) NOT NULL,
        city_id INT,

        CONSTRAINT FK_zip_city FOREIGN KEY(city_id)
            REFERENCES cities(city_id)
            ON DELETE CASCADE
    );
    """
cursor = connection.cursor()
cursor.execute(create_zip_table_query)

In [ ]:
def insert_values_zip():
    for i in range(len(zip_frame)):
        zip_code = zip_frame.iloc[i, 0]
        lat      = zip_frame.iloc[i, 1]
        lng      = zip_frame.iloc[i, 2]
        city_id  = zip_frame.iloc[i, 3]
        insert_query = f"""
            INSERT INTO zip(zip, latitude, longitude, city_id)
            VALUES({zip_code}, {lat}, {lng}, {city_id})
        """
        cursor.execute(insert_query)
insert_values_zip()

In [ ]:
city_id_data.head()

In [ ]:
county_names = city_id_data[['county_name', 'county_fips']]

In [ ]:
for_county = city_id_data[['zip', 'county_weights']]
for_county

In [ ]:
import json
json_dict = json.loads(for_county.iloc[0, 1])
zips = []
fips = []
weight = []
for i in range(len(for_county)):
    zip = for_county.iloc[i, 0]
    json_ = json.loads(for_county.iloc[i, 1])
    for key, value in json_.items():
        zips.append(zip)
        fips.append(key)
        weight.append(value)

In [ ]:
counties = pd.DataFrame({
    "zip": zips,
    "fips": fips,
    "weight": weight
})
counties

In [ ]:
counties['fips'] = counties['fips'].astype(int)
counties = counties.merge(county_names, left_on='fips', right_on='county_fips').drop_duplicates()

In [ ]:
counties

In [ ]:
create_county_table_query = f"""
    CREATE TABLE county (
        county_id SERIAL PRIMARY KEY,
        zip INT,
        county_name VARCHAR(50),
        county_weight FLOAT(7),
        county_fips INT,
        CONSTRAINT FK_zip_county FOREIGN KEY(zip)
            REFERENCES zip(zip)
            ON DELETE CASCADE
    );
    """
cursor = connection.cursor()
cursor.execute(create_county_table_query)

In [ ]:
def insert_values_county():
    for i in range(len(counties)):
        zip_code = counties.iloc[i, 0]
        fips     = counties.iloc[i, 1]
        weight   = counties.iloc[i, 2]
        county_name = counties.iloc[i, 3]
        insert_query = f"""
            INSERT INTO county(zip, county_name, county_weight, county_fips)
            VALUES({zip_code}, '{county_name.replace("'", "''")}', {weight}, {fips})
        """
        cursor.execute(insert_query)
insert_values_county()

# So the database is created, I have following relations
## cities = S(CityId(Primary Key), city_name, state_id, state_name, population(pop), density)
Here I chose to create CityId as a primary key for each city, as one city has more than one zip code associated with it, so spearating it will be one of the steps to satisfy 3rd Normalized form of the database
## county = S(County_id(Primary Key), zip(Foreign key refrencing to zips), county_name, county_weight, county_fips)
Here again I separated counties for a dataset for one reason to keep notmalised form, and second reason to distribute weights of fips into each zip. In json structure I had key of fips and weight, which represent the division of authority of each fip. So for each key value pair I createdseparate row in this table, with zip id solving the normalisation issue and giving optimal solution
## zips = S(zip(Primary Key), latitude, longitude, city_id)
Here I put zip code as primary key, as It's unique in my structure. I have city_id as foregin key conecting to cities table.

### It's much more optimal structure to keep this data. It gives much convenient mechanisms to access data with less memory and time complexity. Domains for each structure are specified in queries above

In [ ]:
problem_3_a_query = f"""
    SELECT z.zip, c.state_name, c.city_name
    FROM zip z
        INNER JOIN cities c
            ON z.city_id = c.city_id
    WHERE c.city_name  LIKE 'Honolulu' OR c.state_name LIKE 'Hawaii'
    """
cursor = connection.cursor()
cursor.execute(problem_3_a_query)

In [89]:
problem_3_b_query = f"""
    SELECT DISTINCT state_name
    FROM cities
    """
cursor = connection.cursor()
cursor.execute(problem_3_b_query)

In [90]:
problem_3_c_query = f"""
    SELECT zip, c.state_name, c2.county_name
    FROM zip z
        INNER JOIN cities c on z.city_id = c.city_id
        INNER JOIN county c2 on z.zip = c2.zip
    WHERE c.density > 300
      AND c.pop> 31234
    """
cursor = connection.cursor()
cursor.execute(problem_3_b_query)